In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

# cnn model vary kernel size
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam


In [ ]:
df = pd.read_csv('/content/pamap2.csv').drop('Unnamed: 0', axis =1)
df

,timestamp,activityID,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,...,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,ankleOrientation1,ankleOrientation2,ankleOrientation3,ankleOrientation4,subject_id
0,37.70,1,100.0,30.3750,2.301060,7.25857,6.09259,2.207200,7.24042,5.95555,...,-0.048878,-0.006328,-60.2040,-37.1225,-57.8847,1.000000,0.000000,0.000000,0.000000,1
1,37.81,1,100.0,30.3750,2.246150,7.48180,5.55219,2.251300,7.40595,5.65313,...,-0.017912,0.015243,-60.7820,-36.2116,-58.4891,1.000000,0.000000,0.000000,0.000000,1
2,37.92,1,100.0,30.3750,2.300000,7.10681,6.09309,2.346130,7.46611,6.12103,...,-0.011798,0.011387,-61.4531,-36.2983,-58.4845,1.000000,0.000000,0.000000,0.000000,1
3,38.03,1,100.0,30.3750,2.494550,7.52335,6.17157,2.591060,7.76653,6.27112,...,-0.009233,0.005823,-62.0065,-36.7217,-58.3608,1.000000,0.000000,0.000000,0.000000,1
4,38.14,1,101.0,30.3750,2.716540,8.30596,4.78671,2.595960,7.94641,4.86635,...,-0.042562,-0.009289,-60.4311,-37.5412,-59.0013,1.000000,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118511,3629.66,5,195.0,30.8125,0.565086,2.59595,9.27677,0.660681,2.70904,9.44024,...,-0.002287,-0.017737,-17.6912,12.0079,26.5782,0.164776,0.798149,0.227263,0.533065,8
118512,3629.77,5,195.0,30.8125,0.570697,2.82459,9.39155,0.495537,2.75557,9.47051,...,-0.007246,0.007512,-17.6928,12.0076,26.4364,0.166393,0.797223,0.228965,0.533221,8
118513,3629.87,5,195.0,30.8125,0.756704,2.78452,9.31749,0.781122,2.72325,9.40988,...,0.003818,0.017649,-17.9459,11.7619,26.1521,0.166735,0.797039,0.229771,0.533041,8
118514,3629.99,5,195.0,30.8125,0.530504,2.67279,9.35297,0.540382,2.87591,9.30409,...,-0.003649,-0.002610,-18.2103,12.4200,26.5759,0.168936,0.795763,0.231222,0.533628,8


In [ ]:
hands_col = []
chest_col = []
for i in df.columns:
  if 'hand' in i and 'Orientation' not in i and 'Temperature' not in i:
    hands_col.append(i)
  elif 'chest' in i and 'Orientation' not in i and 'Temperature' not in i:
    chest_col.append(i)
hands_col.append('activityID')
chest_col.append('activityID')

In [ ]:
len(hands_col), len(chest_col)

(13, 13)

In [ ]:
hand_df = df[hands_col].reset_index()
chest_df = df[chest_col].reset_index()
hand_df

,index,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,handGyro2,handGyro3,handMagne1,handMagne2,handMagne3,activityID
0,0,2.301060,7.25857,6.09259,2.207200,7.24042,5.95555,-0.069961,-0.018328,0.004582,9.15626,-67.18250,-20.0857,1
1,1,2.246150,7.48180,5.55219,2.251300,7.40595,5.65313,-0.431227,0.002685,-0.062964,9.14612,-67.39360,-20.5508,1
2,2,2.300000,7.10681,6.09309,2.346130,7.46611,6.12103,0.075692,-0.030792,0.005246,9.69163,-67.08980,-21.2481,1
3,3,2.494550,7.52335,6.17157,2.591060,7.76653,6.27112,-0.259058,-0.267895,-0.038533,9.58694,-67.08820,-20.8997,1
4,4,2.716540,8.30596,4.78671,2.595960,7.94641,4.86635,0.377115,-0.023688,-0.020670,8.59622,-67.14860,-20.1947,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118511,118511,0.565086,2.59595,9.27677,0.660681,2.70904,9.44024,0.039195,0.012003,-0.026437,17.71230,-4.80473,-41.8342,5
118512,118512,0.570697,2.82459,9.39155,0.495537,2.75557,9.47051,0.011127,0.038150,0.059271,18.33580,-5.21004,-41.4393,5
118513,118513,0.756704,2.78452,9.31749,0.781122,2.72325,9.40988,0.000674,0.034051,0.008866,17.71400,-4.92892,-41.8335,5
118514,118514,0.530504,2.67279,9.35297,0.540382,2.87591,9.30409,-0.048145,0.034565,-0.001782,18.08540,-4.82465,-41.8349,5


In [ ]:
timestep = 40
step = 10

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:


def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)


X_train, y_train = create_dataset(hand_df.drop('activityID' , axis =1),  hand_df['activityID'], timestep,step)
X_test, y_test = create_dataset(chest_df.drop('activityID' , axis =1), chest_df['activityID'], timestep, step)

print(X_train.shape, y_train.shape)

(11848, 40, 13) (11848, 1)


In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
print("X_train.shape: ", X_train.shape)
print("X_test.shape ", X_test.shape)
print("y_train.shape ", y_train.shape)
print("y_test.shape ", y_test.shape)

X_train.shape:  (11848, 40, 13)
X_test.shape  (11848, 40, 13)
y_train.shape  (11848, 14)
y_test.shape  (11848, 14)


In [ ]:
print("X_train.shape: ", X_train.shape)
print("X_test.shape ", X_test.shape)
print("y_train.shape ", y_train.shape)
print("y_test.shape ", y_test.shape)

X_train.shape:  (11848, 40, 13)
X_test.shape  (11848, 40, 13)
y_train.shape  (11848, 14)
y_test.shape  (11848, 14)


In [ ]:
#Epoch 500

verbose,epochs,batch_size=1,10,32

n_timesteps,n_features,n_outputs=X_train.shape[1],X_train.shape[2],y_train.shape[1]

In [ ]:
n_timesteps, n_features, n_outputs

(40, 13, 14)

# Hand -> Hand/Chest

In [ ]:
#CNN MODEL

inputs=keras.Input(shape=(n_timesteps,n_features))

conv_1=tf.keras.layers.Conv1D(filters=64,kernel_size=5,strides=2,activation='relu')(inputs)
maxpool_1=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(conv_1)

conv_2=tf.keras.layers.Conv1D(filters=96,kernel_size=3,strides=1,activation='relu')(maxpool_1)
avg_pooling=tf.keras.layers.GlobalAveragePooling1D()(conv_2)
batch_norm=tf.keras.layers.BatchNormalization()(avg_pooling)

output=tf.keras.layers.Dense(n_outputs,activation='softmax')(batch_norm)
model=tf.keras.Model(inputs=inputs,outputs=output)
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 40, 13)]          0         
                                                                 
 conv1d_14 (Conv1D)          (None, 18, 64)            4224      
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 9, 64)             0         
 g1D)                                                            
                                                                 
 conv1d_15 (Conv1D)          (None, 7, 96)             18528     
                                                                 
 global_average_pooling1d_7  (None, 96)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 batch_normalization_7 (Bat  (None, 96)                384 

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_split = 0.1, shuffle = True)

Epoch 1/110
334/334 [==============================] - 4s 8ms/step - loss: 2.2463 - accuracy: 0.1724 - val_loss: 3.3518 - val_accuracy: 0.0000e+00
Epoch 2/110
334/334 [==============================] - 2s 5ms/step - loss: 2.0305 - accuracy: 0.1831 - val_loss: 2.1167 - val_accuracy: 0.2346
Epoch 3/110
334/334 [==============================] - 2s 5ms/step - loss: 2.0223 - accuracy: 0.1758 - val_loss: 2.0801 - val_accuracy: 0.0000e+00
Epoch 4/110
334/334 [==============================] - 2s 5ms/step - loss: 2.0192 - accuracy: 0.1840 - val_loss: 2.2648 - val_accuracy: 0.0000e+00
Epoch 5/110
334/334 [==============================] - 2s 5ms/step - loss: 2.0228 - accuracy: 0.1829 - val_loss: 2.0750 - val_accuracy: 0.2346
Epoch 6/110
334/334 [==============================] - 2s 5ms/step - loss: 2.0274 - accuracy: 0.1817 - val_loss: 2.0177 - val_accuracy: 0.2186
Epoch 7/110
334/334 [==============================] - 2s 5ms/step - loss: 2.0231 - accuracy: 0.1848 - val_loss: 1.9852 - val_accu

In [ ]:
base_loss,base_accuracy=model.evaluate(X_train, y_train,batch_size=batch_size,verbose=1)

score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

371/371 [==============================] - 1s 2ms/step - loss: 1.4027 - accuracy: 0.5847
Accuracy >58.465564
Base Loss >1.40


In [ ]:
base_loss,base_accuracy=model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)

score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

371/371 [==============================] - 1s 2ms/step - loss: 3.0282 - accuracy: 0.3570
Accuracy >35.702229
Base Loss >3.03


# Chest -> Chest/Hand

In [ ]:
#CNN MODEL

inputs=keras.Input(shape=(n_timesteps,n_features))

conv_1=tf.keras.layers.Conv1D(filters=64,kernel_size=5,strides=2,activation='relu')(inputs)
maxpool_1=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(conv_1)

conv_2=tf.keras.layers.Conv1D(filters=96,kernel_size=3,strides=1,activation='relu')(maxpool_1)
avg_pooling=tf.keras.layers.GlobalAveragePooling1D()(conv_2)
batch_norm=tf.keras.layers.BatchNormalization()(avg_pooling)

output=tf.keras.layers.Dense(n_outputs,activation='softmax')(batch_norm)
model=tf.keras.Model(inputs=inputs,outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 13)]          0         
                                                                 
 conv1d_6 (Conv1D)           (None, 30, 64)            4224      
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 15, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_7 (Conv1D)           (None, 13, 96)            18528     
                                                                 
 global_average_pooling1d_3  (None, 96)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 batch_normalization_3 (Bat  (None, 96)                384 

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
history = model.fit(X_test, y_test, epochs = 100, batch_size = 32, validation_split = 0.1, shuffle = True)

Epoch 1/100
53/53 [==============================] - 2s 10ms/step - loss: 2.7237 - accuracy: 0.1495 - val_loss: 6.4053 - val_accuracy: 0.1935
Epoch 2/100
53/53 [==============================] - 0s 6ms/step - loss: 2.4440 - accuracy: 0.1658 - val_loss: 9.8148 - val_accuracy: 0.0000e+00
Epoch 3/100
53/53 [==============================] - 0s 6ms/step - loss: 2.2299 - accuracy: 0.1838 - val_loss: 3.6857 - val_accuracy: 0.0000e+00
Epoch 4/100
53/53 [==============================] - 0s 6ms/step - loss: 2.1128 - accuracy: 0.1820 - val_loss: 7.6659 - val_accuracy: 0.0000e+00
Epoch 5/100
53/53 [==============================] - 0s 7ms/step - loss: 2.0730 - accuracy: 0.1832 - val_loss: 2.6580 - val_accuracy: 0.0000e+00
Epoch 6/100
53/53 [==============================] - 0s 6ms/step - loss: 2.0486 - accuracy: 0.1688 - val_loss: 3.6717 - val_accuracy: 0.0000e+00
Epoch 7/100
53/53 [==============================] - 0s 6ms/step - loss: 2.0406 - accuracy: 0.1820 - val_loss: 3.8681 - val_accuracy:

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
base_loss,base_accuracy=model.evaluate(X_test, y_test,batch_size=batch_size,verbose=1)

score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

In [ ]:
base_loss,base_accuracy=model.evaluate(X_train,y_train,batch_size=batch_size,verbose=1)

score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))